In [1]:
import numpy as np
import tensorflow as tf
from my_utils import load_mnist_data
from matplotlib import pyplot as plt
from time import time

In [2]:
train_images, train_labels, test_images, test_labels = load_mnist_data()

In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [4]:
learning_rate = 1e-1
loss_history = []

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.placeholder(tf.float32, [None, 10])

y_ = tf.matmul(x, W) + b

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

start = time()
# <--
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(50):
        loss_val, _ = sess.run([loss, train_step], feed_dict={x: train_images, y: train_labels})
        loss_history.append(loss_val)
    # -->    
    finish = time()
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
    print(f'{accuracy_val}%')

print(f'it tooks {finish - start} sec')
plt.plot(loss_history, label='loss')
plt.legend()
plt.draw()

0.9139999747276306%
it tooks 14.618759870529175 sec


In [5]:
learning_rate = 1e-1
loss_history = []
m = 60000
mini_batch_size = 128

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.placeholder(tf.float32, [None, 10])

y_ = tf.matmul(x, W) + b

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

start = time()
# <--
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(50):
        rand_indicies = np.random.choice(m, mini_batch_size, replace=True)
        x_batch = train_images[rand_indicies, :]
        y_batch = train_labels[rand_indicies, :]

        loss_val, _ = sess.run([loss, train_step], feed_dict={x: x_batch, y: y_batch})
        loss_history.append(loss_val)
    # -->    
    finish = time()
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
    print(f'{accuracy_val}%')

print(f'it tooks {finish - start} sec')
plt.plot(loss_history, label='loss')
plt.legend()
plt.draw()

0.8633000254631042%
it tooks 0.17219901084899902 sec


In [9]:
learning_rate = 1e-3
loss_history = []
m = 60000
epoch = 10
mini_batch_size = 128

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784], name='x')
h1 = tf.layers.dense(inputs=x, units=256, activation=tf.nn.relu, use_bias=True, name='h1')
h2 = tf.layers.dense(inputs=h1, units=256, activation=tf.nn.relu, use_bias=True, name='h2')
y_ = tf.layers.dense(inputs=h2, units=10, activation=None, use_bias=True, name='y_')
y = tf.placeholder(tf.float32, [None, 10], name='y')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

start = time()
# <--
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    indices = np.arange(0, m)
    
    for i in range(epoch):
        np.random.shuffle(indices)
        xs = train_images[indices, :]
        ys = train_labels[indices, :]
        
        for j in range(m // mini_batch_size):
            batch_xs = xs[j * mini_batch_size : (j+1) * mini_batch_size, :]
            batch_ys = ys[j * mini_batch_size : (j+1) * mini_batch_size, :]
            loss_val, _ = sess.run([loss, train_step], feed_dict={x: batch_xs, y: batch_ys})
            loss_history.append(loss_val)

    saver = tf.train.Saver()
    saver.save(sess, './models/mnist_nn')
    # -->    
    finish = time()
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
    print(f'{accuracy_val}%')

print(f'it tooks {finish - start} sec')
plt.plot(loss_history, label='loss')
plt.legend()
plt.draw()

0.965399980545044%
it tooks 42.58586382865906 sec
